In [1]:
import json
import spacy
nlp = spacy.load("en_core_web_sm")

In [2]:
### LOAD ###

#keyphrases = [json.loads(ln)["keyphrase"] for ln in open("keyphrases.jsonl")]
data = [json.loads(ln) for ln in open("../../data/train_cmv_cleaned.jsonl")]
#topics = [json.loads(ln) for ln in open("topic_signatures.jsonl")]

In [3]:
import sys
sys.path.append("../utils/")

import keyphrase_extraction
#import utils.keyphrase_extraction

ModuleNotFoundError: No module named 'keyphrase_vectorizers'

In [ ]:
# https://aclanthology.org/E17-1024.pdf
# TODOs: Polarity Shifters
# TODOs: Sentence Levels over the entire argument
# MANUAL: Added "least"
from spacy.matcher import PhraseMatcher
from keyphrase_extraction import exctract_keyphrase
import random

#id = random.randint(0, 2000)
# keyphrase = nlp(keyphrases[id][0].lower())
#topic = nlp(topics[id]["topic_signature"][0])
#arg = nlp(data[id]["argument"])

# TODOs: Sentence Level 
# TODOs: Argument Level
# TODOs: Manage Adjectival Complement 'adcomp'
# TODOs: Sentiment Shifters - reverse polarity of Sentiment Words; k trailing words from Sentiment Shifters

phrase_matcher = PhraseMatcher(nlp.vocab)

# id = 1586

id = random.randint(0, 1000)
claim =  nlp(data[id]["titles"])
arg = nlp(data[id]["arguments"])

# Aspect
keyphrase = exctract_keyphrase(str(arg), n_gram=3)[0]
keyphrase = nlp(keyphrase)

### SENTIMENT LEXICONS ###
pos = [w.replace("\n", "") for w in open("../../data/lexicon/positive_lex.txt")]
neg = [w.replace("\n", "") for w in open("../../data/lexicon/negative_lex.txt")]

compound_word = ""
for i in keyphrase:
    if i.pos_ in ["NOUN", "PROPN"]:
        comps = "".join([str(j) for j in i.children if j.dep_ == "compound"])
        if comps:
            compound_word = comps + " " + str(i)

# match pattern
patterns = [nlp(compound_word)]
phrase_matcher.add("phrases", None, *patterns)

start = 0
stop = 0
matched_phrases = phrase_matcher(claim)
for i, j, k in matched_phrases:
    start = j
    stop = k

pos_score = 0.0
neg_score = 0.0

for idx, tok in enumerate(arg):

    if idx == start or idx == stop - 1:
        continue

    # Polarity Shift
    # NEAR parameter, k
    k = 5
    if tok.dep_ == "neg":
        if tok.text in pos:
            # Shift to Negative
            if idx <= k:
                if idx < start: neg_score += 1/(start - idx)
                else: neg_score += 1/(idx - stop)**0.5
    
        if str(tok.head.text) in neg:
            # Shift to Positive
            if idx < start: pos_score += 1/(start - idx)
            else: pos_score += 1/(idx - stop)**0.5

    if str(tok.text) in pos:
        if idx < start: pos_score += 1/(start - idx)
        else: pos_score += 1/(idx - stop)**0.5

    if str(tok.text) in neg:
        if idx < start: neg_score += 1/(start - idx)
        else: neg_score += 1/(idx - stop)**0.5

result = pos_score - neg_score /(pos_score - neg_score + 1)
stance = ""

neg_score, pos_score
stance = {"arg": arg, "stance": "PRO", "aspect": keyphrase} if result > 0 else {"claim": claim, "stance": "CON", "aspect": keyphrase}

stance

{'arg': These people are definitely victims but in my opinion referring to them as survivors is in many cases overly dramatic. They might be referred to as survivors if their lives legitimately came into jeopardy if they were held at gunpoint if they were severely physically assaulted etc. but most instances of rape unwanted sexual advances isolated to a single instance do not come attached with a serious possibility of death. Rape is definitely one of the worst things that can happen to somebody but I think that survivor suggests that what happened to them was on the same level as living through a genocidal campaign a terminal illness a plane crash or something else of the sort. The former is bad but the latter are almost certainly worse. Rape victims calling themselves survivors takes attention away from more serious issues in which death is on the line. Again rape is a very serious issue but not as serious as issues that consistently result in death.,
 'stance': 'PRO',
 'aspect': ra

In [ ]:
### SENTIMENT LEXICONS ###
pos = [w.replace("\n", "") for w in open("../../data/lexicon/positive_lex.txt")]
neg = [w.replace("\n", "") for w in open("../../data/lexicon/negative_lex.txt")]

def sentence_stance(sentence):
    keyphrase = exctract_keyphrase(str(claim), n_gram=3)[0]
    keyphrase = nlp(keyphrase)

    compound_word = ""
    for i in keyphrase:
        if i.pos_ in ["NOUN", "PROPN"]:
            comps = "".join([str(j) for j in i.children if j.dep_ == "compound"])
            if comps:
                compound_word = comps + " " + str(i)

    pos_score = 0.0
    neg_score = 0.0

    for idx, tok in enumerate(claim):

        if idx == start or idx == stop - 1:
            continue

        # Polarity Shift
        # NEAR parameter, k
        k = 5
        if tok.dep_ == "neg":
            if tok.text in pos:
                # Shift to Negative
                if idx <= k:
                    if idx < start: neg_score += 1/(start - idx)
                    else: neg_score += 1/(idx - stop)**0.5
        
            if str(tok.head.text) in neg:
                # Shift to Positive
                if idx < start: pos_score += 1/(start - idx)
                else: pos_score += 1/(idx - stop)**0.5

        if str(tok.text) in pos:
            if idx < start: pos_score += 1/(start - idx)
            else: pos_score += 1/(idx - stop)**0.5

        if str(tok.text) in neg:
            if idx < start: neg_score += 1/(start - idx)
            else: neg_score += 1/(idx - stop)**0.5

    result = pos_score - neg_score /(pos_score - neg_score + 1)
    stance = ""

    neg_score, pos_score
    stance = {"claim": claim, "stance": "PRO", "aspect": keyphrase} if result > 0 else {"claim": claim, "stance": "CON", "aspect": keyphrase}

    return stance

id = random.randint(0, 1000)
claim =  nlp(data[id]["titles"])

sentence_stance(claim)
arg = nlp(data[id]["arguments"])

stances = []
for i in arg.sents:
    stances.append(sentence_stance(str(i)))

score = 0
for i in stances:
    if i["stance"] == "PRO":
        score += 1
    
    elif i["stance"] == "CON":
        score -= 1

# Add a main Aspect and Topic Here
stances, score

([{'claim': I believe society causes more trauma for rape victims,
   'stance': 'CON',
   'aspect': society causes trauma},
  {'claim': I believe society causes more trauma for rape victims,
   'stance': 'CON',
   'aspect': society causes trauma},
  {'claim': I believe society causes more trauma for rape victims,
   'stance': 'CON',
   'aspect': society causes trauma},
  {'claim': I believe society causes more trauma for rape victims,
   'stance': 'CON',
   'aspect': society causes trauma},
  {'claim': I believe society causes more trauma for rape victims,
   'stance': 'CON',
   'aspect': society causes trauma},
  {'claim': I believe society causes more trauma for rape victims,
   'stance': 'CON',
   'aspect': society causes trauma},
  {'claim': I believe society causes more trauma for rape victims,
   'stance': 'CON',
   'aspect': society causes trauma},
  {'claim': I believe society causes more trauma for rape victims,
   'stance': 'CON',
   'aspect': society causes trauma},
  {'clai

In [ ]:
# from bertopic import BERTopic
# import scipy
# from sentence_transformers import SentenceTransformer

# topic_model = BERTopic()
# #sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
# # topic_model = BERTopic(embedding_model=sentence_model)

# arg = [i["arguments"] for i in data]

# topics, probs = topic_model.fit_transform(arg)

In [4]:
import pardata
data = pardata.load_dataset('wikipedia_category_stance')

ModuleNotFoundError: No module named 'pardata'

In [5]:
data

[{'id': 't3_2ro9ux',
  'titles': 'Anything that is manmade is natural.',
  'arguments': 'I cant remember the topic that spurred this discussion but a friend and I were debating whether manmade things were natural. He took the position that they are unnatural. He cited this definition by MerriamWebster existing in nature and not made or caused by people coming from nature as his basis for the distinction for natural vs. unnatural.However I respectfully disagree with his position and furthermore that definition of natural. People arise from nature. Humankinds capacity to create problemsolve analyze rationalize and build also come from natural processes. How are the things we create unnatural? It is only through natural occurrences that we have this ability why is it that we would give the credit of these things solely to man as opposed to nature? We are not separate from nature thus how can any of our actions or creations be unnatural? If we were somehow separate from nature I would unde

In [21]:
# import requests # External dependency: pip install requests
# import tarfile

# # Downloading the dataset
# url_base = 'https://dax.cdn.appdomain.cloud/dax-wikipedia-category-stance'
# version = '1.0.2'
# fname = 'wikipedia-category-stance.tar.gz'
# url = "{}/{}/{}".format(url_base, version, fname)
# r = requests.get(url)

# if not r.ok:
#     print("There are some errors when downloading {}".format(url))

ConnectionError: HTTPSConnectionPool(host='dax.cdn.appdomain.cloud', port=443): Max retries exceeded with url: /dax-wikipedia-category-stance/1.0.2/wikipedia-category-stance.tar.gz (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x2af711630>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [20]:
id = random.randint(0, 1000)
arg = nlp(data[id]["arguments"])
arg

American patriots have a general mentality against immigration. Saying these people shouldnt be allowed to become part of the country is the same as saying they are worse because they were born on a different side of a line and americans are better because of the side of the line they are born on. This is prominent in many ads and political champagnes namely the slogan Creating jobs for americans. I understand why politics use this slogan because they are trying to get americans to vote for them but this slogan is also prominent in ads made by private corporations. As if creating jobs for americans is morally superior to creating jobs for people of other countries. The companies launching these ads may be trying to win in the american market so they can sell more of their product but the fact that this can increase sales shows that many americans hold the being born on one side of a line belief. I am not blaming the politicians or corporations running ads running these slogans they are

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

mname = "cristian-popa/bart-tl-ng"
tokenizer = AutoTokenizer.from_pretrained(mname)
model = AutoModelForSeq2SeqLM.from_pretrained(mname)

input = "site web google search website online internet social content user"
enc = tokenizer(input, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
outputs = model.generate(
    input_ids=enc.input_ids,
    attention_mask=enc.attention_mask,
    max_length=15,
    min_length=1,
    do_sample=False,
    num_beams=25,
    length_penalty=1.0,
    repetition_penalty=1.5
)

decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded)

KeyboardInterrupt: 

In [32]:
### TOPICS ###
# https://towardsdatascience.com/topic-modeling-with-bert-779f7db187e6
# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('distilbert-base-nli-mean-tokens')
# embeddings = model.encode(data, show_progress_bar=True)

array([0.        , 0.30126753, 0.        , ..., 0.        , 1.        ,
       0.44818607])

In [50]:
### CONCEPT NET ###
# import requests

# obj = requests.get(f"http://api.conceptnet.io/c/en/{stance['aspect']}").json()
# obj.keys()

# for i in obj["edges"]:
#     if i["start"]["language"] == "en":
#         print("start", i["start"])
#         print("rel", i["rel"])
#         print("rel", i["rel"])
#         print(i["start"]["label"])
#         print("rel", i["rel"]["label"])


In [239]:
# import graphviz
# from graphviz import Source
# from nltk import Tree
# import deplacy

# # sentence = 'The brown fox is quick and he is jumping over the lazy dog'
# # doc = nlp(sentence)

# dot = graphviz.Digraph(comment='The Round Table')
# dot.node('A', 'King Arthur')  # doctest: +NO_EXE
# dot.node('B', 'Sir Bedevere the Wise')
# dot.node('L', 'Sir Lancelot the Brave')

# dot.edges(['AB', 'AL'])
# dot.edge('B', 'L', constraint='false')

# print(dot.source)
# dot.render('doctest-output/round-table.gv').replace('\\', '/')

// The Round Table
digraph {
	A [label="King Arthur"]
	B [label="Sir Bedevere the Wise"]
	L [label="Sir Lancelot the Brave"]
	A -> B
	A -> L
	B -> L [constraint=false]
}



'doctest-output/round-table.gv.pdf'

In [255]:
# doc = nlp("The quick brown fox jumps over the lazy dog.")

# dot = graphviz.Digraph(comment='Test')
# def to_tree(node):
#     if node.n_lefts + node.n_rights > 0:
#         print(node.orth_, node.dep_, node.pos_, [i for i in node.children])
#         # dot.edge([to_tree(i) for i in node.children])
#         # [to_tree(i) for i in node.children]
#     else:node.orth_,
#         dot.node(node.orth_)

# for sent in doc.sents:
#     to_tree(sent.root)

jumps ROOT VERB [fox, over, .]
fox nsubj NOUN [The, quick, brown]


TypeError: edge() missing 1 required positional argument: 'head_name'

In [259]:
# import spacy
# from nltk import Tree

# doc = nlp("its the questions we can't answer that teach us the most.")

# def to_nltk_tree(node):
#     if node.n_lefts + node.n_rights > 0:
#         return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
#     else:
#         return node.orth_


# [to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]

                questions                    
  __________________|________________         
 |   |   |          |              teach     
 |   |   |          |           _____|____    
 |   |   |        answer       |     |   most
 |   |   |    ______|______    |     |    |   
its the  .   we     ca    n't that   us  the 



[None]

In [129]:
# doc = nlp("Autonomous cars and magic wands shift insurance liability toward manufacturers")
# interesting_token = doc[7] # or however you identify the token you want
# for noun_chunk in doc.noun_chunks:
#     if interesting_token in noun_chunk:
#         print(noun_chunk)

insurance liability


In [218]:
# for chunk in sent_.noun_chunks:
#     #print(chunk.text)
#     if chunk.text in str(pattern):
#         print(chunk.text, [i for i in chunk.lefts])

Humans are going to die out because of an economic collapse
an economic collapse []


In [ ]:
# ### SENTIMENT LEXICONS ###
# pos = [w.replace("\n", "") for w in open("positive_lex.txt")]
# neg = [w.replace("\n", "") for w in open("negative_lex.txt")]

# id = 2
# claim =  data[id]["titles"] + data[id]["arguments"]
# doc = nlp(claim.lower())
#
# pattern = nlp(keyphrases[id][0].lower())
#
# compound_word = ""
# for i in pattern:
#     if i.pos_ in ["NOUN", "PROPN"]:
#         comps = "".join([str(j) for j in i.children if j.dep_ == "compound"])
#         if comps:
#             compound_word = comps + " " + str(i)
#
#
#
# opinion_sents = []
#
# for i in doc.sents:
#     if str(compound_word) in str(i):
#         opinion_sents.append(i)
#
# stance_distances = {}
# for i, sent in enumerate(opinion_sents):
#     stance_distances[i] = {k: [] for k in sent}
#
#
# # test = [i for i in opinion_sents[0]]
# # for tok in test:
# #     if str(tok.text) in list(str(stance_distances[i].keys())):
# #         print(tok.text)
# #token.text in list(stance_distances[i].keys()):
#
# for i, sent in enumerate(opinion_sents):
#
#     for count, token in enumerate(sent):
#         print(count, token)
#
#         if str(token.text) in list(str(l) for l in stance_distances[i].keys()):
#             pass
#             # if str(token.text) in pos:
#             #     stance_distances[i][token.text].append(("POS", count))
# #
#             # if token.text in neg:
#             #     # stance_distances[i][token.text] = ("NEG", count)
#             #     stance_distances[i][token.text].append(("NEG", count))
#
# #             # else: stance_distances[i][token.text] = ("UNK", count)
# #
#
# print(stance_distances)
#
# # print("benefits" in pos)
# # print(pos)

In [ ]:
# id = 2
# kp = str(" ".join([i for i in keyphrases[id]]))
# kp_doc = nlp(kp)
#
# kp_nouns = []
# kp_nouns.append(" ".join([str(tok) for tok in kp_doc if tok.pos_ in ('NOUN','PROPN')]))
# kp_nouns.append(" ".join(([str(tok) for tok in kp_doc if tok.dep_ == 'compound'])))
#
# kp_nouns = " ".join(kp_nouns)
# print(kp_nouns)
#
# pos = [w for w in open("positive_lex.txt")]
# neg = [w for w in open("negative_lex.txt")]
#
# adj_ = []
# stance = 0
#
# for i in noun_adj_pairs:
#
#     j, k = i
#
#     if j.text in kp_nouns:
#
#         if j.text in pos: stance = 1
#         if j.text in neg: stance = -1
#
#         else: stance = "UNK"
#
#         adj_.append((k, stance))

In [ ]:
# id = 50
# sent_ = data[id]["titles"] + data[id]["arguments"]
# doc = nlp(sent_)
#
# # Consider Childeren == compound etc
# noun_adj_pairs = []
# for i,token in enumerate(doc):
#
#     if token.pos_ not in ('NOUN','PROPN'):
#         continue
#
#     for j in range(i+1,len(doc)):
#         if doc[j].pos_ == 'ADJ' or doc[j].pos_ == "CCONJ":
#             noun_adj_pairs.append((token,doc[j]))
#             break
#
# noun_adj_pairs

In [ ]:
# parsed_sentence = nlp(u'This is my sentence')
# [(token.text,token.i) for token in parsed_sentence]

In [ ]:
# pos = [w.replace("\n", "") for w in open("positive_lex.txt")]
# neg = [w.replace("\n", "") for w in open("negative_lex.txt")]
#
# id = 2
# claim =  data[id]["titles"] + data[id]["arguments"]
# doc = nlp(claim.lower())
#
# pattern = nlp(keyphrases[id][0].lower())
# print(pattern)
#
# compound_word = ""
# for i in pattern:
#     if i.pos_ in ["NOUN", "PROPN"]:
#         comps = "".join([str(j) for j in i.children if j.dep_ == "compound"])
#         if comps:
#             compound_word = comps + " " + str(i)
#
# compound_word

In [ ]:
# Identify the targets of the given topic, claim
# Identify the polarity (sentiment) towards each of the targets



In [132]:
# from spacy.symbols import nsubj, VERB
#
# doc = nlp("Autonomous cars shift insurance liability toward manufacturers")
#
# verbs = set()
# for possible_subject in doc:
#     if possible_subject.dep == nsubj and possible_subject.head.pos == VERB:
#         verbs.add(possible_subject.head)

In [ ]:
# OPEN-IE

# IMPORTANT NOTE; Command: java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000
# TODOs: Use openie library, StanfordOpenIE

# import nltk
# from pycorenlp import *
# import json
# import collections
#
# nlp = StanfordCoreNLP("http://localhost:9000/")
# properties = {
#     "annotators": "tokenize, ssplit, pos, depparse, natlog,openie",
#     "outputFormat": "json",
# }
#
# output = []
# sub_pred_obj = []
# titles = []

# for j in data[13:14]:
#     result = json.loads(nlp.annotate(j["titles"], properties=properties))
#     raw = result["sentences"][0]["openie"]
#     if raw:
#         output.append([j["titles"], result["sentences"][0]["openie"]])
#     else:
#         continue
#
# # for i in output:
# #     sub_pred_obj.append({
# #         "title": i[1],
# #         "subject": i[3][0]["subject"],
# #         "predicate": i[3][0]["relation"],
# #         "object": i[3][0]["object"]
# #     })
#
# for i in sub_pred_obj:
#     print(i)

In [250]:
# Nice Implementation

# texts =  ("I believe that the Gay Marriage discussion isnt as important",
#          )
# docs = nlp.pipe(texts)
#
# compounds = []
# for doc in docs:
#     compounds.append({doc.text:[doc[tok.i:tok.head.i+1] for tok in doc if tok.dep_=="compound"]})
# print(compounds)

[{'I believe that the Gay Marriage discussion isnt as important': [Gay Marriage, Marriage discussion]}]
